# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [61]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [63]:
#!pip install pandas

In [64]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [65]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [66]:
!pip install langchain_community
!pip install -U langchain langchain-openai

In [67]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [68]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None)


In [69]:
prompt = ChatPromptTemplate.from_template("Write a marketing description for the following product: {product}")

In [70]:

chain = LLMChain(llm=llm, prompt=prompt)

In [71]:
product = "smart watch"
chain.run(product)

'Stay connected and on top of your health and fitness goals with our innovative smart watch. This sleek and stylish device not only tells time, but also tracks your steps, monitors your heart rate, and sends you notifications when you receive calls or messages. With a variety of customizable watch faces and interchangeable bands, you can personalize your smart watch to fit your unique style. Say goodbye to constantly checking your phone - let our smart watch keep you organized and motivated throughout your busy day. Elevate your wrist game and upgrade to a smart watch today!'

## SimpleSequentialChain

In [72]:
from langchain.chains import SimpleSequentialChain

In [73]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write a short creative product description for: {input}"

)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [74]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a tweet based on this product description: {input}"

)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [75]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [76]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Introducing the latest must-have accessory for the modern tech-savvy individual - the smart watch. This sleek and stylish device combines the convenience of a traditional watch with the advanced features of a smartphone. From tracking your fitness goals to receiving notifications and calls right on your wrist, this smart watch is the ultimate companion for life on the go. Stay connected, organized, and in control with the touch of a button. Upgrade your wrist game with the smart watch and take your everyday tasks to the next level.
Elevate your wrist game with the latest must-have accessory - the smart watch! Stay connected and in control on the go with this sleek and stylish device that combines the convenience of a traditional watch with the advanced features of a smartphone. #techsavvy #smartwatch

> Finished chain.


'Elevate your wrist game with the latest must-have accessory - the smart watch! Stay connected and in control on the go with this sleek and stylish device that combines the convenience of a traditional watch with the advanced features of a smartphone. #techsavvy #smartwatch'

**Repeat the above twice for different products**

## SequentialChain

In [77]:
from langchain.chains import SequentialChain

In [78]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into Arabic: {input}"
)

chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="translated_review"  # Give a name to your output
)

In [79]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize this review: {translated_review}"
)

chain_two = LLMChain(
    llm=llm, 
    prompt=second_prompt, 
    output_key="summary" # Give a name to your output
)

In [80]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate this summary into French: {summary}"
)
chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="french_translation"  
)


In [81]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow-up message based on this summary: {summary} and its French translation: {french_translation}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup")


In [82]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["input"],
    output_variables=["translated_review", "summary", "french_translation", "followup"],
    verbose=True
)

In [83]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'input': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'translated_review': 'أجد الطعم سيء. الرغوة لا تدوم ، هذا غريب. أشتري نفس المنتجات في المتاجر والطعم أفضل بكثير ... هل هو دفعة قديمة أم تقليد؟',
 'summary': 'The reviewer found the taste to be bad and the foam did not last long, which they found strange. They usually buy the same products in stores and the taste is much better. They question whether it is an old batch or a knock-off product.',
 'french_translation': "Le critique a trouvé le goût mauvais et la mousse n'a pas duré longtemps, ce qui l'a trouvé étrange. Ils achètent généralement les mêmes produits en magasin et le goût est bien meilleur. Ils se demandent s'il s'agit d'un vieux lot ou d'un produit contrefait.",
 'followup': "Thank you for sharing your feedback on the product. It's disappointing to hear that you found the taste to be bad and the foam

**Repeat the above twice for different products or reviews**

## Router Chain

In [84]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [85]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [86]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [87]:
llm = ChatOpenAI(temperature=0)

In [88]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [89]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [90]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [91]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [92]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [93]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases."

In [94]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [95]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [96]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(temperature=0.7)


prompt_translate = ChatPromptTemplate.from_template(
    "Translate this review to English: {input}"
)
chain_translate = LLMChain(llm=llm, prompt=prompt_translate, output_key="english_review")


prompt_sentiment = ChatPromptTemplate.from_template(
    "What is the sentiment of this review? (positive, negative, neutral): {english_review}"
)
chain_sentiment = LLMChain(llm=llm, prompt=prompt_sentiment, output_key="sentiment")


prompt_response = ChatPromptTemplate.from_template(
    "Write a friendly customer support message for a {sentiment} review."
)
chain_response = LLMChain(llm=llm, prompt=prompt_response, output_key="response")


sentiment_chain = SequentialChain(
    chains=[chain_translate, chain_sentiment, chain_response],
    input_variables=["input"],
    output_variables=["response"],
    verbose=True
)


example_review = "الخدمة كانت بطيئة جدًا وما وصلت الطلبية في الوقت المتوقع"
result = sentiment_chain.run({"input": example_review})
print("\nFinal Response:\n", result)



> Entering new SequentialChain chain...

> Finished chain.

Final Response:
 Hello,

We're sorry to hear that you had a negative experience with our product/services. We strive to provide the best possible experience for our customers and we apologize if we fell short of your expectations. 

We would love the opportunity to make things right and address any issues you may have encountered. Please reach out to our customer support team so we can assist you further and hopefully turn your experience around. 

Thank you for bringing this to our attention and we appreciate your feedback as it helps us improve our products/services for all of our customers. 

Warm regards,
[Your Name]
Customer Support Team
